In [1]:

import os
import numpy as np
import matplotlib as mpl
from glob import glob
import matplotlib.pyplot as plt

import parsl
from parsl.app.app import python_app, bash_app
from parsl.data_provider.files import File
from parslpw import pwconfig,pwargs

parsl.load(pwconfig)
print("pwconfig loaded")


loading local pwconfig
pwargs: Namespace(f='/root/.local/share/jupyter/runtime/kernel-4afa62fd-3df7-4792-98ce-829aba0e3b06.json')
pwconfig loaded


In [82]:
@bash_app
def run_nwchem(stdout='nwchem.stdout', stderr='nwchem.stderr', inputs=[], outputs=[]):
    import os
    label = os.path.splitext(os.path.basename(outputs[0]))[0]
    run_script = os.path.basename(inputs[0])
    docker = os.path.basename(inputs[2])

    sed_command = "sed -i -e 's,_XYZFILE_,%s.xyz,' -e 's/_NAME_/%s/' %s"%(os.path.splitext(os.path.basename(inputs[1]))[0],label,os.path.basename(inputs[0]))
    nwchem_command = "/bin/bash " + docker + " " + run_script + " > " + os.path.basename(outputs[0])
    
    return '''
        %s 
        %s
        mkdir -p outputs/out
        mkdir -p outputs/db
        mv *.db outputs/db
        mv *.out outputs/out
    ''' % (sed_command,nwchem_command)


In [86]:
pwargs = {
    'nw_file':'/pw/storage/inputs/dft_631g.nw',
    'structures':'/pw/storage/gcs_store/gw100_structures/100-41-4.xyz',
    'png_file':'/pw/storage/20191101-162428.png',
}

prefix = 'file://parslhost/'
run_file = File(prefix+os.getcwd()+"/run_nwchem_vanilla.sh ")
nw_file = File(prefix+pwargs['nw_file'])
xyz_files = pwargs['structures'].split('---')

runs=[]
for xyz_file_raw in xyz_files:
    xyz_file = File(prefix+xyz_file_raw)
    out_file = File(prefix+os.getcwd()+'/./outputs/out/'+xyz_file.filename.replace('.xyz','.out'))
    db_file = File(prefix+os.getcwd()+'/./outputs/db/'+xyz_file.filename.replace('.xyz','.db'))
    r = run_nwchem(inputs=[nw_file,xyz_file,run_file], outputs=[out_file,db_file])
    runs.append(r)

print("Running",len(runs),"simulations...")
[print(r.result()) for r in runs]


in PWRsync stage_in: file.path=//pw/storage/inputs/dft_631g.nw
in PWRsync stage_in: file=<<class 'parsl.data_provider.files.File'> at 0x7ff2f41ca160 url=file://parslhost//pw/storage/inputs/dft_631g.nw scheme=file netloc=parslhost path=/pw/storage/inputs/./dft_631g.nw filename=dft_631g.nw local_path=./dft_631g.nw>, curdir=/pw/workflows/nwchem_energy_molecule_set_v1 working_dir=/tmp/parsllogs pid=1520
in PWRsync stage_in: file=<<class 'parsl.data_provider.files.File'> at 0x7ff2f41ca160 url=file://parslhost//pw/storage/inputs/dft_631g.nw scheme=file netloc=parslhost path=/pw/storage/inputs/./dft_631g.nw filename=dft_631g.nw local_path=./dft_631g.nw> pid=1520
in PWRsync stage_in: file.path=//pw/storage/gcs_store/gw100_structures/100-41-4.xyz
in PWRsync stage_in: file=<<class 'parsl.data_provider.files.File'> at 0x7ff2f41c6278 url=file://parslhost//pw/storage/gcs_store/gw100_structures/100-41-4.xyz scheme=file netloc=parslhost path=/pw/storage/gcs_store/gw100_structures/./100-41-4.xyz filen

[None]